In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import copy

### 그림그리는 함수

In [2]:
def show_v_table_small(v_table, env):
    for i in range(env.reward.shape[0]):        
        print("+----------"*env.reward.shape[1])
        print("|", end="")
        for j in range(env.reward.shape[1]):
            print("{0:8.2f}  |".format(v_table[i,j]),end="")
        print()
    print("+----------"*env.reward.shape[1])

# V table 그리기    
def show_v_table(v_table, env):    
    for i in range(env.reward.shape[0]):        
        print("+-----------------"*env.reward.shape[1],end="")
        print("+")
        for k in range(3):
            print("|",end="")
            for j in range(env.reward.shape[1]):
                if k==0:
                    print("                 |",end="")
                if k==1:
                        print("   {0:8.2f}      |".format(v_table[i,j]),end="")
                if k==2:
                    print("                 |",end="")
            print()
    print("+-----------------"*env.reward.shape[1],end="")
    print("+")
    
# Q table 그리기
def show_q_table(q_table,env):
    for i in range(env.reward.shape[0]):
        print("+-----------------"*env.reward.shape[1],end="")
        print("+")
        for k in range(3):
            print("|",end="")
            for j in range(env.reward.shape[1]):
                if k==0:
                    print("{0:10.2f}       |".format(q_table[i,j,0]),end="")
                if k==1:
                    print("{0:6.2f}    {1:6.2f} |".format(q_table[i,j,3],q_table[i,j,1]),end="")
                if k==2:
                    print("{0:10.2f}       |".format(q_table[i,j,2]),end="")
            print()
    print("+-----------------"*env.reward.shape[1],end="")
    print("+")
    

# 정책 policy 화살표로 그리기
def show_q_table_arrow(q,env):
    for i in range(env.reward.shape[0]):        
        print("+-----------------"*env.reward.shape[1],end="")
        print("+")
        for k in range(3):
            print("|",end="")
            for j in range(env.reward.shape[1]):
                if k==0:
                    if np.max(q[i,j,:]) == q[i,j,0]:
                        print("        ↑       |",end="")
                    else:
                        print("                 |",end="")
                if k==1:                    
                    if np.max(q[i,j,:]) == q[i,j,1] and np.max(q[i,j,:]) == q[i,j,3]:
                        print("      ←  →     |",end="")
                    elif np.max(q[i,j,:]) == q[i,j,1]:
                        print("          →     |",end="")
                    elif np.max(q[i,j,:]) == q[i,j,3]:
                        print("      ←         |",end="")
                    else:
                        print("                 |",end="")
                if k==2:
                    if np.max(q[i,j,:]) == q[i,j,2]:
                        print("        ↓       |",end="")
                    else:
                        print("                 |",end="")
            print()
    print("+-----------------"*env.reward.shape[1],end="")
    print("+")    
    
# 정책 policy 화살표로 그리기
def show_policy_small(policy,env):
    for i in range(env.reward.shape[0]):        
        print("+----------"*env.reward.shape[1],end="")
        print("+")
        print("|", end="")
        for j in range(env.reward.shape[1]):
            if env.reward_list1[i][j] == "road":
                if policy[i,j] == 0:
                    print("   ↑     |",end="")
                elif policy[i,j] == 1:
                    print("   →     |",end="")
                elif policy[i,j] == 2:
                    print("   ↓     |",end="")
                elif policy[i,j] == 3:
                    print("   ←     |",end="")
            else:
                print("          |",end="")
        print()
    print("+----------"*env.reward.shape[1],end="")
    print("+")
    
# 정책 policy 화살표로 그리기
def show_policy(policy,env):
    for i in range(env.reward.shape[0]):        
        print("+-----------------"*env.reward.shape[1],end="")
        print("+")
        for k in range(3):
            print("|",end="")
            for j in range(env.reward.shape[1]):
                if k==0:
                    print("                 |",end="")
                if k==1:
                    if policy[i,j] == 0:
                        print("      ↑         |",end="")
                    elif policy[i,j] == 1:
                        print("      →         |",end="")
                    elif policy[i,j] == 2:
                        print("      ↓         |",end="")
                    elif policy[i,j] == 3:
                        print("      ←         |",end="")
                if k==2:
                    print("                 |",end="")
            print()
    print("+-----------------"*env.reward.shape[1],end="")
    print("+")

### Agent 구현

In [3]:
class Agent():
    
    # 1. 행동에 따른 에이전트의 좌표 이동(위, 오른쪽, 아래, 왼쪽) 
    action = np.array([[-1,0],[0,1],[1,0],[0,-1]])
    
    # 2. 각 행동별 선택확률
    select_action_pr = np.array([0.25,0.25,0.25,0.25])
    
    # 3. 에이전트의 초기 위치 저장
    def __init__(self):
        self.pos = (0,0)
    
    # 4. 에이전트의 위치 저장
    def set_pos(self,position):
        self.pos = position
        return self.pos
    
    # 5. 에이전트의 위치 불러오기
    def get_pos(self):
        return self.pos

### Environment 구현

In [4]:
class Environment():
    
    # 1. 미로밖(절벽), 길, 목적지와 보상 설정
    cliff = -3
    road = -1
    goal = 1
    
    # 2. 목적지 좌표 설정
    goal_position = [2,2]
    
    # 3. 보상 리스트 숫자
    reward_list = [[road,road,road],
                   [road,road,road],
                   [road,road,goal]]
    
    # 4. 보상 리스트 문자
    reward_list1 = [["road","road","road"],
                    ["road","road","road"],
                    ["road","road","goal"]]
    
    # 5. 보상 리스트를 array로 설정
    def __init__(self):
        self.reward = np.asarray(self.reward_list)    

    # 6. 선택된 에이전트의 행동 결과 반환 (미로밖일 경우 이전 좌표로 다시 복귀)
    def move(self, agent, action):
        
        done = False
        
        # 6.1 행동에 따른 좌표 구하기
        new_pos = agent.pos + agent.action[action]
        
        # 6.2 현재좌표가 목적지 인지확인
        if self.reward_list1[agent.pos[0]][agent.pos[1]] == "goal":
            reward = self.goal
            observation = agent.set_pos(agent.pos)
            done = True
        # 6.3 이동 후 좌표가 미로 밖인 확인    
        elif new_pos[0] < 0 or new_pos[0] >= self.reward.shape[0] or new_pos[1] < 0 or new_pos[1] >= self.reward.shape[1]:
            reward = self.cliff
            observation = agent.set_pos(agent.pos)
            done = True
        # 6.4 이동 후 좌표가 길이라면
        else:
            observation = agent.set_pos(new_pos)
            reward = self.reward[observation[0],observation[1]]
            
        return observation, reward, done

### ϵ-greedy 확률 변화
입실론-그리디 방법 (입실론-탐욕정책)

학습 과정에서 ϵ 만큼의 확률로는 무작위 행동을 취하고, 1-ϵ 만큼의 확률로는 그리디하게 행동을 선택함
다음은 행동 a4가 최적일 때 입실론에 따른 각 행동의 확률 변화.

In [5]:
policy = np.array([0.0, 0.0, 0.0, 1.0])
print("episilon |   a1  |    a2   |    a3  |   a4  ")
for epsilon in reversed(range(0, 11)):
    epsilon /= 10
    up = epsilon / len(policy)
    right = epsilon / len(policy)
    down = epsilon / len(policy)
    # a4가 최적의 정책이라고 생각함
    left = (1 -  epsilon) + (epsilon / len(policy))
    print("{0:}      | {1:.3f} |  {2:.3f}  |  {3:.3f} |  {4:.3f}".format(np.round(epsilon,3),np.round(up,3),np.round(right,3),np.round(down,3),np.round(left,3)))


episilon |   a1  |    a2   |    a3  |   a4  
1.0      | 0.250 |  0.250  |  0.250 |  0.250
0.9      | 0.225 |  0.225  |  0.225 |  0.325
0.8      | 0.200 |  0.200  |  0.200 |  0.400
0.7      | 0.175 |  0.175  |  0.175 |  0.475
0.6      | 0.150 |  0.150  |  0.150 |  0.550
0.5      | 0.125 |  0.125  |  0.125 |  0.625
0.4      | 0.100 |  0.100  |  0.100 |  0.700
0.3      | 0.075 |  0.075  |  0.075 |  0.775
0.2      | 0.050 |  0.050  |  0.050 |  0.850
0.1      | 0.025 |  0.025  |  0.025 |  0.925
0.0      | 0.000 |  0.000  |  0.000 |  1.000


### ϵ-greedy 를 이용한 몬테카를로 Control

In [6]:
def generate_episode_with_policy(env, agent, policy):
    #★★★★★★★★
    # 재밌는점: gamma를 평소처럼 크게 잡으면 이상한 일이...?
    #★★★★★★★★
    gamma = 0.75
    # 에피소드를 저장할 리스트
    episode = []
    # 이전에 방문여부 체크
    
    # 에이전트는 항상 (0,0)에서 출발
    i = 0
    j = 0
    agent.set_pos([i,j])    
    #에피소드의 수익을 초기화
    #감쇄율의 지수
    max_step = 100
    # 에피소드 생성
    for k in range(max_step):
        pos = agent.get_pos()        
        # 현재 상태의 정책을 이용해 행동을 선택한 후 이동
        action = np.random.choice(range(0,len(agent.action)), p=policy[pos[0],pos[1],:]) 
        observaetion, reward, done = env.move(agent, action)    
        
        # 방문 이력 저장(상태, 행동, 보상, 0값 (G가 채워질 예정) ) 
        episode.append([pos,action,reward, 0])     

        # 에피소드가 종료했다면 루프에서 탈출
        if done == True:                
            break        
            
    G = 0
    # episode 순간마다 G값 구하기
    for ep_i in range(len(episode)-1, -1, -1):
        G = G*gamma + episode[ep_i][2] # + reward
        episode[ep_i][3] = G
    
    return i, j, G, episode

# first visit 를 사용할지 every visit를 사용할 지 결정
# first_visit = True : first visit
# first_visit = False : every visit
for first_visit in [True, False]:
    if first_visit:
        print("start first visit MC")
    else : 
        print("start every visit MC")
    print()

    # ϵ-정책을 이용하는 몬테카를로 control 알고리즘
    np.random.seed(0)
    # 환경, 에이전트를 초기화
    env = Environment()
    agent = Agent()

    # 모든 𝑠∈𝑆,𝑎∈𝐴(𝑆)에 대해 초기화:
    # # 𝑄(𝑠,𝑎)←임의의 값 (행동 개수, 미로 세로, 미로 가로)
    Q_table = np.random.rand(env.reward.shape[0], env.reward.shape[1],len(agent.action))
    print("Initial Q(s,a)")
    show_q_table(Q_table,env)

    # 상태를 방문한 횟수를 저장하는 테이블
    Q_visit = np.zeros((env.reward.shape[0], env.reward.shape[1],len(agent.action)))

    # 미로 모든 상태에서 최적 행동을 저장하는 테이블
    # 각 상태에서 Q 값이 가장 큰 행동을 선택 후 optimal_a 에 저장
    optimal_a = np.zeros((env.reward.shape[0],env.reward.shape[1]))
    for i in range(env.reward.shape[0]):
        for j in range(env.reward.shape[1]):
            optimal_a[i,j] = np.argmax(Q_table[i,j,:])
    print("initial optimal_a")
    show_policy(optimal_a,env)

    # π(𝑠,𝑎)←임의의 𝜖−탐욕 정책
    # 무작위로 행동을 선택하도록 지정
    policy = np.zeros((env.reward.shape[0], env.reward.shape[1],len(agent.action)))

    # 한 상태에서 가능한 확률의 합이 1이 되도록 계산
    epsilon = 0.9
    for i in range(env.reward.shape[0]):
        for j in range(env.reward.shape[1]):
            for k in range(len(agent.action)):
                if optimal_a[i,j] == k:
                    policy[i,j,k] = 1 - epsilon + epsilon/len(agent.action)
                else:
                    policy[i,j,k] = epsilon/len(agent.action)
    print("Initial Policy")
    show_q_table(policy,env)


    # 최대 에피소드 수를 지정
    max_episode = 10000



    for epi in tqdm(range(max_episode)):
    # for epi in range(max_episode):

        # π를 이용해서 에피소드 1개를 생성
        x,y,G,episode = generate_episode_with_policy(env, agent, policy)
        visit = np.zeros((env.reward.shape[0], env.reward.shape[1]))
        for step_num in range(len(episode)):
            # episode[step_num][0][0] : step_num번째 방문한 상태의 x 좌표
            # episode[step_num][0][1] : step_num번째 방문한 상태의 y 좌표
            # episode[step_num][1] : step_num번째 상태에서 선택한 행동
            # episode[step_num][3] : step_num번째 상태에서 얻은 반환값
            i = episode[step_num][0][0]
            j = episode[step_num][0][1]
            action = episode[step_num][1]
            G = episode[step_num][3]
            
            if first_visit and visit[i][j] == 1:
                continue
            visit[i][j] = 1
            # 에피소드 시작점을 카운트
            Q_visit[i,j,action] += 1
            
            # Incremental mean : 𝑄(𝑠,𝑎)←𝑎𝑣𝑒𝑟𝑎𝑔𝑒(𝑅𝑒𝑡𝑢𝑟𝑛(𝑠,𝑎)) 
            Q_table[i,j,action] += 1 / Q_visit[i,j,action]*(G-Q_table[i,j,action])

        # (c) 에피소드 안의 각 s에 대해서 :
        # 미로 모든 상태에서 최적 행동을 저장할 공간 마련
        # 𝑎∗ ←argmax_a 𝑄(𝑠,𝑎)
        for i in range(env.reward.shape[0]):
            for j in range(env.reward.shape[1]):
                optimal_a[i,j] = np.argmax(Q_table[i,j,:])            

        # 모든 𝑎∈𝐴(𝑆) 에 대해서 :
        # 새로 계산된 optimal_a 를 이용해서 행동 선택 확률 policy (π) 갱신
        epsilon = 0.9*(1 - epi/max_episode)

        for i in range(env.reward.shape[0]):
            for j in range(env.reward.shape[1]):
                for k in range(len(agent.action)):
                    if optimal_a[i,j] == k:
                        policy[i,j,k] = 1 - epsilon + epsilon/len(agent.action)
                    else:
                        policy[i,j,k] = epsilon/len(agent.action)

    print("Final Q(s,a)")
    show_q_table(Q_table,env)
    print("Final policy")
    show_q_table(policy,env)
    print("Final optimal_a")
    show_policy(optimal_a,env)

start first visit MC

Initial Q(s,a)
+-----------------+-----------------+-----------------+
|      0.55       |      0.42       |      0.96       |
|  0.54      0.72 |  0.89      0.65 |  0.53      0.38 |
|      0.60       |      0.44       |      0.79       |
+-----------------+-----------------+-----------------+
|      0.57       |      0.02       |      0.98       |
|  0.09      0.93 |  0.87      0.83 |  0.78      0.80 |
|      0.07       |      0.78       |      0.46       |
+-----------------+-----------------+-----------------+
|      0.12       |      0.52       |      0.46       |
|  0.94      0.64 |  0.77      0.41 |  0.62      0.57 |
|      0.14       |      0.26       |      0.02       |
+-----------------+-----------------+-----------------+
initial optimal_a
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      →         |      ←         |      ↑         |
|                 |                 |               

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:04<00:00, 2479.50it/s]


Final Q(s,a)
+-----------------+-----------------+-----------------+
|     -3.00       |     -3.00       |     -3.00       |
| -3.00     -2.97 | -3.27     -2.32 | -2.99     -3.00 |
|     -2.31       |     -2.63       |     -0.99       |
+-----------------+-----------------+-----------------+
|     -3.18       |     -3.17       |     -2.41       |
| -3.00     -2.58 | -3.13     -1.03 | -2.44     -3.00 |
|     -1.36       |     -1.77       |      1.75       |
+-----------------+-----------------+-----------------+
|     -2.56       |     -1.92       |      1.00       |
| -3.00     -0.05 | -2.02      1.75 |  1.00      1.00 |
|     -3.00       |     -3.00       |      1.00       |
+-----------------+-----------------+-----------------+
Final policy
+-----------------+-----------------+-----------------+
|      0.00       |      0.00       |      0.00       |
|  0.00      0.00 |  0.00      1.00 |  0.00      0.00 |
|      1.00       |      0.00       |      1.00       |
+-----------------+---

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:04<00:00, 2233.22it/s]

Final Q(s,a)
+-----------------+-----------------+-----------------+
|     -3.00       |     -3.00       |     -3.00       |
| -3.00     -2.91 | -3.24     -2.62 | -3.10     -3.00 |
|     -2.23       |     -1.93       |     -1.23       |
+-----------------+-----------------+-----------------+
|     -3.06       |     -2.69       |     -2.32       |
| -3.00     -1.37 | -2.71     -1.00 | -2.48     -3.00 |
|     -2.33       |     -0.24       |      1.75       |
+-----------------+-----------------+-----------------+
|     -3.01       |     -1.88       |      1.00       |
| -3.00     -0.73 | -1.93      1.75 |  1.00      1.00 |
|     -3.00       |     -3.00       |      1.00       |
+-----------------+-----------------+-----------------+
Final policy
+-----------------+-----------------+-----------------+
|      0.00       |      0.00       |      0.00       |
|  0.00      0.00 |  0.00      0.00 |  0.00      0.00 |
|      1.00       |      1.00       |      1.00       |
+-----------------+---

### TD(0) prediction

TD로 V(s)를 구해보자

In [7]:
# TD(0) prediction
np.random.seed(0)
# 환경, 에이전트를 초기화
env = Environment()
agent = Agent()
gamma = 0.9

#초기화 : 
#π← 평가할 정책
# 가능한 모든 행동이 무작위로 선택되도록 지정
#𝑉← 임의의 상태가치 함수
V = np.zeros((env.reward.shape[0], env.reward.shape[1]))

# 최대 에피소드, 에피소드의 최대 길이를 지정
max_episode = 10000
max_step = 100

alpha = 0.01

print("start TD(0) prediction")

# 각 에피소드에 대해 반복 :
for epi in tqdm(range(max_episode)):
    delta =0
    # s 를 초기화
    i = 0
    j = 0
    agent.set_pos([i,j])

    #  에피소드의 각 스텝에 대해 반복 :
    for k in range(max_step):
        pos = agent.get_pos()
        # a←상태 𝑠 에서 정책 π에 의해 결정된 행동 
        # 가능한 모든 행동이 무작위로 선택되게 함
        action = np.random.randint(0,4)
        # 행동 a 를 취한 후 보수 r과 다음 상태 s’를 관측
        # s←𝑠'
        observation, reward, done = env.move(agent,action)
        # V(𝑠)←V(𝑠)+ α[𝑟+𝛾𝑉(𝑠^)−𝑉(𝑠)]
        V[pos[0],pos[1]] += alpha * (reward + gamma * V[observation[0],observation[1]] - V[pos[0],pos[1]])
        # s가 마지막 상태라면 종료
        if done == True:
            break
            
print("V(s)")
show_v_table(np.round(V,2),env)

start TD(0) prediction


100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 14615.30it/s]

V(s)
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|     -12.59      |     -11.01      |     -10.12      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|     -10.85      |      -8.53      |      -5.80      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -9.68      |      -6.06      |       3.48      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+


## 모든 코드를 실행해 보고 출력 결과를 얻어보자.